In [ ]:
from spectral_cube import SpectralCube

In [ ]:
import pyspeckit

In [3]:
cube = SpectralCube.read('/orange/adamginsburg/ALMA_IMF/2017.1.01355.L/RestructuredImagingResults/G351.77/B6/fullcubes_12m/G351.77_B6_spw7_12M_spw7.image.fits')

Cube is a Stokes cube, returning spectral cube for I component


In [4]:
import sys
sys.executable

'/orange/adamginsburg/miniconda3/envs/xclass/bin/python'

In [5]:
im = cube[50].value

In [6]:
mxy,mxx = np.unravel_index(np.nanargmax(im), cube.shape[1:])
mxy,mxx

(467, 565)

In [ ]:
from astropy import units as u

In [ ]:
csp = cube[:,mxy,mxx].to(u.K, u.brightness_temperature(csp.spectral_axis))

In [ ]:
sp = pyspeckit.Spectrum.from_hdu(csp.hdu)

In [ ]:
sp.plotter()

In [ ]:
from pyspeckit.spectrum.models import lte_molecule

In [ ]:
from astropy import units as u

In [ ]:
freqs, aij, deg, EU, partfunc = lte_molecule.get_molecular_parameters('CH3OCHO', fmin=200*u.GHz, fmax=250*u.GHz, export_limit=1e5)

In [ ]:
mod = lte_molecule.generate_model(sp.xarr, 0*u.km/u.s, 5*u.km/u.s, 50*u.K, 1e16*u.cm**-2, freqs, aij, deg, EU, partfunc)

In [ ]:
sp.plotter(clear=True)
sp.plotter.axis.plot(sp.xarr, mod)

In [ ]:
#Vso... re:xclass - for line ID things, we can probably do this with pyspeckit, and I understand what's going on a lot better there.  For example.